<a href="https://colab.research.google.com/github/weso500/IOT-Anomaly-Detection/blob/main/Single_Image_VAE_C_FLUX_GenLatent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Obtained intially from https://keras.io/examples/generative/vae/ and https://colab.research.google.com/github/keras-team/keras-io/blob/master/examples/generative/ipynb/vae.ipynb

In [1]:
pip install --upgrade keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.1 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1


In [2]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import numpy as np
import tensorflow as tf
import keras
from keras import ops
from keras import layers

In [3]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.seed_generator = keras.random.SeedGenerator(1337)

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = ops.shape(z_mean)[0]
        dim = ops.shape(z_mean)[1]
        epsilon = keras.random.normal(shape=(batch, dim), seed=self.seed_generator)
        return z_mean + ops.exp(0.5 * z_log_var) * epsilon

Use this notebook as example for obtaining images: https://colab.research.google.com/drive/1EOFDisibXb9a4A3lZ_2z4Jbhlnx0OVy6 The below takes care of obtaining images

In [4]:
latent_dim = 3

encoder_inputs = keras.Input(shape=(256, 256, 3))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 256, 256, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 128, 128, 32)   │            896 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 64, 64, 64)     │         18,496 │ conv2d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 262144)         │              0 │ conv2d_1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 128)            │     33,554,560 │ flatten[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ z_mean (Dense)            │ (None, 3)              │            387 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ z_log_var (Dense)         │ (None, 3)              │            387 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sampling (Sampling)       │ (None, 3)              │              0 │ z_mean[0][0],          │
│                           │                        │                │ z_log_var[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 33,574,726 (128.08 MB)

 Trainable params: 33,574,726 (128.08 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(64 * 64 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((64, 64, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(3, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 3)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 262144)              │       1,048,576 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 64, 64, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose (Conv2DTranspose)   │ (None, 128, 128, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_1 (Conv2DTranspose) │ (None, 256, 256, 32)        │          18,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_2 (Conv2DTranspose) │ (None, 256, 256, 3)         │             867 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,104,835 (4.21 MB)

 Trainable params: 1,104,835 (4.21 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
from tensorflow.keras.layers import Layer
class KLLossLayer(Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
        kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))

        return kl_loss

class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        self.kl_loss_layer = KLLossLayer()

    def call(self, inputs):  # Add the call method
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        return reconstructed

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = ops.mean(
                ops.sum(
                    keras.losses.binary_crossentropy(data, reconstruction),
                    axis=(1, 2),
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - ops.square(z_mean) - ops.exp(z_log_var))
            kl_loss = ops.mean(ops.sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        kl_loss = self.kl_loss_layer([z_mean, z_log_var])
        return reconstructed, kl_loss, z_mean, z_log_var

    def predict(self, x):
        reconstructed, kl_loss, z_mean, z_log_var = super().predict(x)
        return reconstructed, kl_loss, z_mean, z_log_var


In [7]:
!unzip /content/Gold_Folder_Final.zip

Streaming output truncated to the last 5000 lines.
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/108Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/109Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/10Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/110Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/111Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/112Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/113Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/114Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/115Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGB

In [8]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/Gold_Folder_Final/Training_Data/Normal/Spectrograms_Normal_C-FLUX',
  image_size=(256, 256),
  batch_size=32,
  label_mode=None)


normalization_layer = layers.Rescaling(scale= 1./255)

normalized_ds = train_ds.map(lambda x: normalization_layer(x))

Found 500 files.


In [9]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C-FLUX',
  image_size=(256, 256),
  shuffle=False,
  batch_size=1,
  label_mode=None)

file_paths = test_ds.file_paths
print(file_paths)

normalization_layer = layers.Rescaling(scale= 1./255)

normalized_test_ds = test_ds.map(lambda x: normalization_layer(x))

Found 190 files.
['/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C-FLUX/500Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C-FLUX/501Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C-FLUX/502Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C-FLUX/503Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C-FLUX/504Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C-FLUX/505Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C-FLUX/506Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C-FLUX/507Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C-FLUX/508Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C-FLUX/509Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C-FLUX/510Normal.png', '/content/Gold

In [10]:
test_anomaly_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/Gold_Folder_Final/Test_Data/Anomaly/Spectrograms_Anomaly_C-FLUX',
  image_size=(256, 256),
  shuffle=False,
  batch_size=1,
  label_mode=None)

file_paths = test_ds.file_paths
print(file_paths)

normalization_layer = layers.Rescaling(scale= 1./255)

normalized_anomaly_test_ds = test_anomaly_ds.map(lambda x: normalization_layer(x))

Found 182 files.
['/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C-FLUX/500Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C-FLUX/501Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C-FLUX/502Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C-FLUX/503Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C-FLUX/504Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C-FLUX/505Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C-FLUX/506Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C-FLUX/507Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C-FLUX/508Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C-FLUX/509Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C-FLUX/510Normal.png', '/content/Gold

In [11]:
#(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
#mnist_digits = np.concatenate([x_train, x_test], axis=0)
#mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(normalized_ds, epochs=15, batch_size=32)

Epoch 1/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 29s 748ms/step - kl_loss: 634.3809 - loss: 43059.3203 - reconstruction_loss: 42424.9414
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 87ms/step - kl_loss: 527.4767 - loss: 24496.8145 - reconstruction_loss: 23969.3379
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - kl_loss: 77.8596 - loss: 23027.2070 - reconstruction_loss: 22949.3457
Epoch 4/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 93ms/step - kl_loss: 59.2850 - loss: 22649.1074 - reconstruction_loss: 22589.8242
Epoch 5/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 94ms/step - kl_loss: 54.6715 - loss: 22417.6152 - reconstruction_loss: 22362.9434
Epoch 6/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - kl_loss: 48.9627 - loss: 22043.4492 - reconstruction_loss: 21994.4883
Epoch 7/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - kl_loss: 42.8289 - loss: 21533.7559 - reconstruction_loss: 21490.9258
Epoch 8/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 92ms/step - kl_loss: 39.8993 - loss: 20633.0898 - reconstruction_loss: 20593.1914
Epoch 9/15
1

In [13]:
train_ds_test_ = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/Gold_Folder_Final/Training_Data/Normal/Spectrograms_Normal_C-FLUX',
  image_size=(256, 256),
  batch_size=1,
  label_mode=None)


normalization_layer = layers.Rescaling(scale= 1./255)

normalized_ds_test1 = train_ds_test_.map(lambda x: normalization_layer(x))

Found 500 files.


In [14]:
pip install sewar

  Preparing metadata (setup.py) ... done
  Created wheel for sewar: filename=sewar-0.4.6-py3-none-any.whl size=11419 sha256=d23f66e08911d7f46b9bf15607fb558ae921f9c1fe41a33250c32227f1daa3f9
  Stored in directory: /root/.cache/pip/wheels/3f/af/02/9c6556ba287b62a945d737def09b8b8c35c9e1d82b9dfae84c
Successfully built sewar


In [15]:
import cv2
from urllib.request import AbstractBasicAuthHandler
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp

#mse_values_normal = [] # Create an empty list to store mse values
#mse_values_anomaly = []
mse_values = []
kldiv1 = []
zmean1 = []
zmean2 = []
zmean3 = []
zlogvar1 = []
zlogvar2 = []
zlogvar3 = []

for elem in normalized_ds_test1.take(500):
  for i in range(len(elem)):
    elem_np = elem.numpy()
    exp = vae.predict(elem)
    exp_np = tf.get_static_value(exp)
    #plt.imshow(np.squeeze(elem_np))
    #plt.show()
    #plt.imshow(np.squeeze(exp_np[0]))
    #plt.show()
    #x = cv2.PSNR(elem_np, exp_np[0])
    x = mse(elem_np, exp_np[0])
    print(x)
    print(exp_np[1])
    kldiv1.append(exp_np[1])
    print(exp_np[2])
    zmean1.append(exp_np[2][0][0])
    zmean2.append(exp_np[2][0][1])
    zmean3.append(exp_np[2][0][2])
    print(exp_np[3])
    zlogvar1.append(exp_np[3][0][0])
    zlogvar2.append(exp_np[3][0][1])
    zlogvar3.append(exp_np[3][0][2])
    mse_values.append(x) # Append each mse value to the list



1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
0.0011034014547831833
58.164448
[[ 4.9686756 -8.257012   3.583733 ]]
[[-4.289358  -4.8981614 -4.398779 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
0.001080750124286204
58.158455
[[ 5.074933  -8.114711   3.7813277]]
[[-4.3157606 -4.63246   -4.4317937]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
0.0015565051060561282
56.816017
[[ 4.7712197 -8.322879   3.2927022]]
[[-4.208047  -5.1957245 -4.317784 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
0.0012751638334644667
58.54415
[[ 5.1026974 -8.128501   3.814292 ]]
[[-4.329801  -4.616303  -4.4485703]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
0.00129321891122919
60.91847
[[ 5.1823936 -8.381647   3.735845 ]]
[[-4.3984857 -4.8449435 -4.4964523]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
0.0011272297158728965
58.10019
[[ 4.947407  -8.273098   3.5535517]]
[[-4.284059  -4.94009   -4.3942194]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
0.0012082425178474873
57.483658
[[ 5.0351005 -8.062256   3.777948 ]]
[[-4.29178   -4.602258  -4.4123416]]


In [16]:
import pandas as pd

dict = {'MSE': mse_values, 'KL Loss': kldiv1, 'Zmean1': zmean1, 'Zmean2': zmean2, 'Zmean3': zmean3, 'Zlogvar1': zlogvar1,'Zlogvar2': zlogvar2, 'Zlogvar3': zlogvar3 }

df = pd.DataFrame(dict)

# saving the dataframe
df.to_csv('C-FLUX_train.csv')

In [17]:
import cv2
from urllib.request import AbstractBasicAuthHandler
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp

#mse_values_normal = [] # Create an empty list to store mse values
#mse_values_anomaly = []
mse_values = []
kldiv1 = []
zmean1 = []
zmean2 = []
zmean3 = []
zlogvar1 = []
zlogvar2 = []
zlogvar3 = []

for elem in normalized_test_ds.take(190):
  for i in range(len(elem)):
    elem_np = elem.numpy()
    exp = vae.predict(elem)
    exp_np = tf.get_static_value(exp)
    #plt.imshow(np.squeeze(elem_np))
    #plt.show()
    #plt.imshow(np.squeeze(exp_np[0]))
    #plt.show()
    #x = cv2.PSNR(elem_np, exp_np[0])
    x = mse(elem_np, exp_np[0])
    print(x)
    print(exp_np[1])
    kldiv1.append(exp_np[1])
    print(exp_np[2])
    zmean1.append(exp_np[2][0][0])
    zmean2.append(exp_np[2][0][1])
    zmean3.append(exp_np[2][0][2])
    print(exp_np[3])
    zlogvar1.append(exp_np[3][0][0])
    zlogvar2.append(exp_np[3][0][1])
    zlogvar3.append(exp_np[3][0][2])
    mse_values.append(x) # Append each mse value to the list

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
0.0022520807651842585
55.740665
[[ 4.2594066 -8.742364   2.359597 ]]
[[-4.041797  -6.1996303 -4.06396  ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
0.0022227945382244734
55.659874
[[ 4.281989  -8.710037   2.4135442]]
[[-4.0487285 -6.13477   -4.07426  ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
0.002117672620482162
56.895164
[[ 4.307495  -8.841597   2.3631146]]
[[-4.0857277 -6.264525  -4.0920095]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
0.0021329950414779625
57.827923
[[ 4.4124937 -8.86195    2.4801145]]
[[-4.135133  -6.185734  -4.1458626]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
0.002356879648606674
55.468487
[[ 4.214859  -8.745793   2.3067925]]
[[-4.026162  -6.2521896 -4.046138 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
0.0024486532487731825
53.60161
[[ 4.119525  -8.59841    2.2704458]]
[[-3.9522607 -6.168873  -3.9841201]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
0.002208589206523856
55.57191
[[ 4.259988  -8.721944   2.3692396]]
[[-4.041796  -6.1738367 -4.058169 

In [18]:
import pandas as pd

dict = {'MSE': mse_values, 'KL Loss': kldiv1, 'Zmean1': zmean1, 'Zmean2': zmean2, 'Zmean3': zmean3, 'Zlogvar1': zlogvar1,'Zlogvar2': zlogvar2, 'Zlogvar3': zlogvar3 }

df2 = pd.DataFrame(dict)

# saving the dataframe
df2.to_csv('C-FLUX_testnorm.csv')

In [19]:
import cv2
from urllib.request import AbstractBasicAuthHandler
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp

#mse_values_normal = [] # Create an empty list to store mse values
#mse_values_anomaly = []
mse_values = []
kldiv1 = []
zmean1 = []
zmean2 = []
zmean3 = []
zlogvar1 = []
zlogvar2 = []
zlogvar3 = []

for elem in normalized_anomaly_test_ds.take(190):
  for i in range(len(elem)):
    elem_np = elem.numpy()
    exp = vae.predict(elem)
    exp_np = tf.get_static_value(exp)
    #plt.imshow(np.squeeze(elem_np))
    #plt.show()
    #plt.imshow(np.squeeze(exp_np[0]))
    #plt.show()
    #x = cv2.PSNR(elem_np, exp_np[0])
    x = mse(elem_np, exp_np[0])
    print(x)
    print(exp_np[1])
    kldiv1.append(exp_np[1])
    print(exp_np[2])
    zmean1.append(exp_np[2][0][0])
    zmean2.append(exp_np[2][0][1])
    zmean3.append(exp_np[2][0][2])
    print(exp_np[3])
    zlogvar1.append(exp_np[3][0][0])
    zlogvar2.append(exp_np[3][0][1])
    zlogvar3.append(exp_np[3][0][2])
    mse_values.append(x) # Append each mse value to the list

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
0.1116282798865495
26.038502
[[-1.7575272 -5.8592157 -2.510382 ]]
[[-0.40693662 -8.52517    -1.5442023 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
0.031355791447115755
30.028404
[[ 0.8729114  -7.032907   -0.76346225]]
[[-2.1573377 -7.5369253 -2.343781 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
0.030676542832916296
29.535366
[[ 2.0515766 -6.784278   0.5457732]]
[[-2.606929  -6.078581  -2.7093203]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
0.0014774453292457515
56.42395
[[ 4.8732586 -8.14191    3.5110645]]
[[-4.223789  -4.878969  -4.3429756]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
0.03193837448606716
28.18342
[[ 1.7231156  -6.6974525   0.23650824]]
[[-2.4439209 -6.3031325 -2.5739744]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
0.03351417006246882
30.470499
[[ 2.1585302  -6.8733053   0.62622505]]
[[-2.666323  -6.077234  -2.7691948]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
0.0011076713319603008
58.51319
[[ 5.115464  -8.107364   3.8390248]]
[[-4.331833  -4.5753107 -4.44

In [20]:
import pandas as pd

dict = {'MSE': mse_values, 'KL Loss': kldiv1, 'Zmean1': zmean1, 'Zmean2': zmean2, 'Zmean3': zmean3, 'Zlogvar1': zlogvar1,'Zlogvar2': zlogvar2, 'Zlogvar3': zlogvar3 }

df3 = pd.DataFrame(dict)

# saving the dataframe
df3.to_csv('C-FLUX_testanom.csv')

In [ ]:
from google.colab import files
files.download('/content/B+FLUX_testanom.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('/content/B+FLUX_testnorm.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('/content/B+_FLUX_train.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>